# G1 scraping
This notebook aims to build a scaper of the news in https://g1.globo.com/

## Setup

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta

from src import scrapers

# Path to geckodriver executable
geckodriver_path = '/snap/bin/firefox.geckodriver'


s = Service(executable_path=geckodriver_path)
# Create a Firefox webdriver instance
# opens a window
driver = webdriver.Firefox(service=s)

In [2]:
g1_scraper = scrapers.G1NewsScraper()
data = g1_scraper.scrap_news(driver)
data

Scroll 1, height=7986
Scroll 2, height=11451
Scroll 3, height=13135
Scroll 4, height=12657
Scroll 5, height=15958
Scroll 6, height=15572
Scroll 7, height=18831
Scroll 8, height=18136
Scroll 9, height=21565
Scroll 10, height=20561


,Title,Time,Theme,Header,Resume
0,"VÍDEO: Em nova gafe, Biden confunde vice Kamal...",2024-07-11 22:07:06.926795,GloboNews em Pauta,None,None
1,"GUGA CHACRA: 'Situação de Biden melhorou, mas ...",2024-07-11 21:59:06.926808,GloboNews em Pauta,None,None
2,'Vidas na rua': números mostram o tamanho do p...,2024-07-11 21:47:06.926812,Vidas na rua,Pessoas sem-teto,None
3,Câmara aprova proposta que pode perdoar multas...,2024-07-11 20:47:06.926815,Política,Emenda constitucional,PEC da Anistia livra de punição legendas que d...
4,PF descobre dispositivos ilegais para roubar d...,2024-07-11 18:47:06.926818,Política,Sistema clandestino,Varredura encontrou instalações clandestinas e...
...,...,...,...,...,...
91,Câmara aprova anistia a partidos que descumpri...,2024-07-11 20:47:06.926984,Jornal Nacional,None,"Deputados aprovaram, em primeiro turno, a Prop..."
92,Prefeitura do Rio anuncia empresa que vai oper...,2024-07-11 19:47:06.926985,Rio de Janeiro,None,Projeto prevê oito linhas obrigatórias de barc...
93,Câmara aprova e envia para sanção projeto de m...,2024-07-11 19:47:06.926987,Política,None,"Texto já havia sido aprovado na Casa, mas volt..."
94,Arroz e ovo quase todo dia: alunos de escolas ...,2024-07-11 19:47:06.926989,Rio de Janeiro,None,"Em uma das escolas do município da Baixada, de..."


## Start

In [2]:
driver.get('https://g1.globo.com/')

In [3]:
# scrolling the page

scrolls = 10

current_height = driver.execute_script("return document.body.scrollHeight")

for i in range(scrolls):
    # scroll to the end of the page
    driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
    sleep(1)
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    
    # if not autoscoll
    if new_height == current_height:
        # click the 'Veja mais' button
        driver.find_element(By.CSS_SELECTOR, value='.load-more > a:nth-child(1)').click()
        
        # repeat the scroll
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
        current_height = driver.execute_script("return document.body.scrollHeight")
    else:
        current_height = new_height
        
    print(f'Scroll {i + 1}, height={current_height}')
    
    # time for the page to load
    sleep(4)

Scroll 1, height=7981
Scroll 2, height=9601
Scroll 3, height=13262
Scroll 4, height=12258
Scroll 5, height=11756
Scroll 6, height=15138
Scroll 7, height=14752
Scroll 8, height=17765
Scroll 9, height=17325
Scroll 10, height=20758


## Scraping the data

In [3]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'lxml')
content_blocks = soup.find_all('div', class_ = '_evg')

In [6]:
highlight_area = soup.find('div', class_ = 'row small-collapse large-uncollapse')

highlighted_news = highlight_area.find_all('ul', class_ = 'bstn-hl-list')
len(highlighted_news)

3

In [6]:

len(content_blocks)

10

In [7]:
for block in content_blocks:
    news_list = block.find_all('div', class_ = 'feed-post-body')
    
    print(len(news_list))

0
0
49
8
9
5
8
6
8
5


In [24]:
titles = []
times = []
themes = []
headers = []
resumes = []
highlight = []


for hnews in highlighted_news:
    title = hnews.find('span', class_ = 'bstn-hl-title gui-color-primary gui-color-hover gui-color-primary-bg-after')
    titles.append(title.text)
    
    theme = hnews.find('span', class_ = 'bstn-hl-chapeu gui-subject gui-color-primary-bg-after')
    themes.append(None if theme is None else theme.text)

    times.append('Há 1 minuto')

    resumes.append(None)
    headers.append(None)
    
    highlight.append(1)
    

for block in content_blocks:
    news_list = block.find_all('div', class_ = 'feed-post-body')
    for news in news_list:
        title = news.find('a', class_ = 'feed-post-link gui-color-primary gui-color-hover')
        titles.append(title.text)
        
        header = news.find('span', 'feed-post-header-chapeu')
        headers.append(None if header is None else header.text)

        time = news.find('span', 'feed-post-datetime')
        times.append(None if time is None else time.text)

        theme = news.find('span', 'feed-post-metadata-section')
        themes.append(None if theme is None else theme.text)

        resume = news.find('div', class_='feed-post-body-resumo')
        resumes.append(None if resume is None else resume.text)
        
        highlight.append(0)

        

data = pd.DataFrame({
    'Title': titles,
    'Time': times,
    'Theme': themes,
    'Header': headers,
    'Resume': resumes,
    'Highlighted': highlight
})

data
        

,Title,Time,Theme,Header,Resume,Highlighted
0,Trump escolhe senador J.D. Vance para ser cand...,Há 1 minuto,Eleições nos EUA,None,None,1
1,VÍDEO: pitbull foge de casa e persegue morador...,Há 1 minuto,Feira de Santana,None,None,1
2,Por que há tantos europeus revoltados com os t...,Há 1 minuto,Protestos e hostilidade,None,None,1
3,Ex-crítico de Trump e autor de livro que virou...,Há 1 hora,Eleições nos EUA 2024,None,None,0
4,Moraes derruba sigilo de áudio de reunião entr...,Há 22 minutos,Blog da Julia Duailibi,Blog da Julia Duailibi,PF diz que então residente discutiu como blind...,0
5,Leia transcrição do diálogo; PF diz que Ramage...,Agora,Política,None,None,0
6,Vídeo mostra momento em que noivo é atropelado...,Há 13 minutos,Rio de Janeiro,Tragédia após casamento,None,0
7,Especialista diz que movimento de cabeça salvo...,Há 60 minutos,Fantástico,Tiro de raspão,None,0
8,VÍDEO: testemunhas alertaram sobre atirador mo...,Há 51 minutos,Eleições nos EUA 2024,None,None,0
9,Brasil chama embaixador na Argentina para disc...,Há 3 horas,Política,Diplomacia,None,0


In [25]:
def convert_to_datetime(time_str):
    """convert "Há X [time unit]" to datetime"""
    if 'hora' in time_str or 'horas' in time_str:
        # Extract the number of hours
        hours_ago = int(time_str.split(' ')[1])
        # Subtract the hours from the current datetime
        return datetime.now() - timedelta(hours=hours_ago)
    elif 'minuto' in time_str or 'minutos' in time_str:
        # Extract the number of minutes
        minutes_ago = int(time_str.split(' ')[1])
        # Subtract the minutes from the current datetime
        return datetime.now() - timedelta(minutes=minutes_ago)
    elif 'dia' in time_str or 'dias' in time_str:
        # Extract the number of days
        days_ago = int(time_str.split(' ')[1])
        # Subtract the days from the current datetime
        return datetime.now() - timedelta(days=days_ago)
    elif 'mês' in time_str or 'meses' in time_str:
        # Extract the number of months
        months_ago = int(time_str.split(' ')[1])
        # Subtract the months from the current datetime
        # Note: This is an approximation as timedelta does not support months directly
        return datetime.now() - timedelta(days=30*months_ago)
    else:
        return None  # If the format does not match, return None

# Apply the function to the series
data['Time'] = data['Time'].apply(convert_to_datetime)

data


,Title,Time,Theme,Header,Resume,Highlighted
0,Trump escolhe senador J.D. Vance para ser cand...,2024-07-15 17:40:07.389414,Eleições nos EUA,None,None,1
1,VÍDEO: pitbull foge de casa e persegue morador...,2024-07-15 17:40:07.389420,Feira de Santana,None,None,1
2,Por que há tantos europeus revoltados com os t...,2024-07-15 17:40:07.389421,Protestos e hostilidade,None,None,1
3,Ex-crítico de Trump e autor de livro que virou...,2024-07-15 16:41:07.389422,Eleições nos EUA 2024,None,None,0
4,Moraes derruba sigilo de áudio de reunião entr...,2024-07-15 17:19:07.389424,Blog da Julia Duailibi,Blog da Julia Duailibi,PF diz que então residente discutiu como blind...,0
5,Leia transcrição do diálogo; PF diz que Ramage...,NaT,Política,None,None,0
6,Vídeo mostra momento em que noivo é atropelado...,2024-07-15 17:28:07.389425,Rio de Janeiro,Tragédia após casamento,None,0
7,Especialista diz que movimento de cabeça salvo...,2024-07-15 16:41:07.389428,Fantástico,Tiro de raspão,None,0
8,VÍDEO: testemunhas alertaram sobre atirador mo...,2024-07-15 16:50:07.389430,Eleições nos EUA 2024,None,None,0
9,Brasil chama embaixador na Argentina para disc...,2024-07-15 14:41:07.389431,Política,Diplomacia,None,0


In [26]:
data

,Title,Time,Theme,Header,Resume,Highlighted
0,Trump escolhe senador J.D. Vance para ser cand...,2024-07-15 17:40:07.389414,Eleições nos EUA,None,None,1
1,VÍDEO: pitbull foge de casa e persegue morador...,2024-07-15 17:40:07.389420,Feira de Santana,None,None,1
2,Por que há tantos europeus revoltados com os t...,2024-07-15 17:40:07.389421,Protestos e hostilidade,None,None,1
3,Ex-crítico de Trump e autor de livro que virou...,2024-07-15 16:41:07.389422,Eleições nos EUA 2024,None,None,0
4,Moraes derruba sigilo de áudio de reunião entr...,2024-07-15 17:19:07.389424,Blog da Julia Duailibi,Blog da Julia Duailibi,PF diz que então residente discutiu como blind...,0
5,Leia transcrição do diálogo; PF diz que Ramage...,NaT,Política,None,None,0
6,Vídeo mostra momento em que noivo é atropelado...,2024-07-15 17:28:07.389425,Rio de Janeiro,Tragédia após casamento,None,0
7,Especialista diz que movimento de cabeça salvo...,2024-07-15 16:41:07.389428,Fantástico,Tiro de raspão,None,0
8,VÍDEO: testemunhas alertaram sobre atirador mo...,2024-07-15 16:50:07.389430,Eleições nos EUA 2024,None,None,0
9,Brasil chama embaixador na Argentina para disc...,2024-07-15 14:41:07.389431,Política,Diplomacia,None,0
